In [8]:
import pandas as pd
import numpy as np

# 모델 선택 및 평가 관련 라이브러리
from sklearn.model_selection import train_test_split #,GridSearchCV(하이퍼파라미터), cross_val_score(교차 검증)
from sklearn.metrics import roc_auc_score, accuracy_score, classification_report
                            #(accuracy는 맞춘 비율을 계산하여 정확도를 높이지만, 모델이 클래스 구분을 잘 했는지는 파악 불가
                            #roc는 확률값을 활용하여 모델의 성능 평가,, 클래스를 잘 구별하는 지 확인, 1에 가까울수록 좋은 모델
                                #accuracy는 데이터가 균형 잡혀 있을 때 G, roc는 데이터가 불균형할 때 G )

# 전처리: 범주형 인코딩, 결측치 대체, 스케일링
from sklearn.preprocessing import OrdinalEncoder       #범주형 데이터 숫자로 변환
from sklearn.preprocessing import StandardScaler       #데이터 표준화하여 평균 0, 표준편차 1로 변환, 데이터 크기 관련X 모델이 동일한 가중치 부여, (SVM, 로지스틱회귀, KNN, PCA 필수)
from sklearn.impute import SimpleImputer               #결측치 자동 대체 (strategy='mean':평균값, 'median':중앙값,'most_frequent':최빈값)


In [9]:
# train.csv, test.csv 불러오기 + 불필요 컬럼 삭제(ID)
train = pd.read_csv('./train.csv', encoding='utf-8').drop(columns=['ID'])
test = pd.read_csv('./test.csv', encoding='utf-8').drop(columns=['ID'])

In [10]:
# 타겟(입신 성공 여부) 분리 + 결측치 제거
y = train['임신 성공 여부']  # 타겟 변수 분리
x = train.drop(columns=['임신 성공 여부'])  # 입력 변수(x) 정의

# x에서 y의 결측치가 있는 행 제거
x = x.loc[y.dropna().index]  
y = y.dropna()  # y에서도 결측치 제거


In [11]:
#범주형 컬럼 전체 정의
categorical_columns = [
    "시술 시기 코드",
    "시술 당시 나이",
    "시술 유형",
    "특정 시술 유형",
    "배란 자극 여부",
    "배란 유도 유형",
    "단일 배아 이식 여부",
    "착상 전 유전 검사 사용 여부",
    "착상 전 유전 진단 사용 여부",
    "남성 주 불임 원인",
    "남성 부 불임 원인",
    "여성 주 불임 원인",
    "여성 부 불임 원인",
    "부부 주 불임 원인",
    "부부 부 불임 원인",
    "불명확 불임 원인",
    "불임 원인 - 난관 질환",
    "불임 원인 - 남성 요인",
    "불임 원인 - 배란 장애",
    "불임 원인 - 여성 요인",
    "불임 원인 - 자궁경부 문제",
    "불임 원인 - 자궁내막증",
    "불임 원인 - 정자 농도",
    "불임 원인 - 정자 면역학적 요인",
    "불임 원인 - 정자 운동성",
    "불임 원인 - 정자 형태",
    "배아 생성 주요 이유",
    "총 시술 횟수",
    "클리닉 내 총 시술 횟수",
    "IVF 시술 횟수",
    "DI 시술 횟수",
    "총 임신 횟수",
    "IVF 임신 횟수",
    "DI 임신 횟수",
    "총 출산 횟수",
    "IVF 출산 횟수",
    "DI 출산 횟수",
    "난자 출처",
    "정자 출처",
    "난자 기증자 나이",
    "정자 기증자 나이",
    "동결 배아 사용 여부",
    "신선 배아 사용 여부",
    "기증 배아 사용 여부",
    "대리모 여부",
    "PGD 시술 여부",
    "PGS 시술 여부"
]

#범주형 컬럼을 문자열(str)로 변환
for col in categorical_columns:
  for col in categorical_columns:
    x.loc[:, col] = x[col].astype(str)
    test.loc[:, col] = test[col].astype(str)     #.loc[] 활용해야 안전

C:\Users\양소정\AppData\Local\Temp\ipykernel_23376\4034366299.py:55: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['1' '0' '1' ... '1' '1' '0']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  x.loc[:, col] = x[col].astype(str)
C:\Users\양소정\AppData\Local\Temp\ipykernel_23376\4034366299.py:56: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['1' '1' '1' ... '0' '0' '1']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  test.loc[:, col] = test[col].astype(str)     #.loc[] 활용해야 안전
C:\Users\양소정\AppData\Local\Temp\ipykernel_23376\4034366299.py:55: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['0.0' '0.0' '0.0' ... '0.0' '0.0' '0.0']' has dtype incompatible with float64, please explicitly cast to a comp

In [12]:
#수치형 컬럼 전체 정의
numeric_columns = [
    "임신 시도 또는 마지막 임신 경과 연수",
    "총 생성 배아 수",
    "미세주입된 난자 수",
    "미세주입에서 생성된 배아 수",
    "이식된 배아 수",
    "미세주입 배아 이식 수",
    "저장된 배아 수",
    "미세주입 후 저장된 배아 수",
    "해동된 배아 수",
    "해동 난자 수",
    "수집된 신선 난자 수",
    "저장된 신선 난자 수",
    "혼합된 난자 수",
    "파트너 정자와 혼합된 난자 수",
    "기증자 정자와 혼합된 난자 수",
    "난자 채취 경과일",
    "난자 해동 경과일",
    "난자 혼합 경과일",
    "배아 이식 경과일",
    "배아 해동 경과일"
]

In [13]:
#train 데이터를 train/val로 분할
x_train, x_val, y_train, y_val = train_test_split(
    x, y, test_size=0.2, random_state=42, stratify=y
)

In [14]:
#범주형 변수 인코딩
ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
x_train_enc = x_train.copy()
x_train_enc[categorical_columns] = ordinal_encoder.fit_transform(x_train_enc[categorical_columns])

x_val_enc   = x_val.copy()
x_val_enc[categorical_columns]   = ordinal_encoder.transform(x_val_enc[categorical_columns])


In [15]:
#수치형 변수 결측치 대체 (중앙값 사용)
imputer = SimpleImputer(strategy='median')
x_train_enc[numeric_columns] = imputer.fit_transform(x_train_enc[numeric_columns])
x_val_enc[numeric_columns]   = imputer.transform(x_val_enc[numeric_columns])


**XGBoost 모델 학습**

In [16]:
from xgboost import XGBClassifier

# n_estimators를 크게 잡고 early_stopping_rounds를 통해 최적의 에포크에서 중단
xgb_model = XGBClassifier(
    n_estimators=200,
    learning_rate=0.1,
    max_depth=6,
    random_state=42,
    eval_metric='logloss'
)

In [18]:
#fit 변수 eval_set, early stopping 사용
xgb_model.fit(
    x_train_enc, y_train,
    eval_set=[(x_val_enc, y_val)],
    #early_stopping_rounds=10,
    verbose=True
)

[0]	validation_0-logloss:0.56093
[1]	validation_0-logloss:0.55169
[2]	validation_0-logloss:0.54390
[3]	validation_0-logloss:0.53727
[4]	validation_0-logloss:0.53168
[5]	validation_0-logloss:0.52684
[6]	validation_0-logloss:0.52262
[7]	validation_0-logloss:0.51890
[8]	validation_0-logloss:0.51568
[9]	validation_0-logloss:0.51283
[10]	validation_0-logloss:0.51032
[11]	validation_0-logloss:0.50812
[12]	validation_0-logloss:0.50613
[13]	validation_0-logloss:0.50447
[14]	validation_0-logloss:0.50295
[15]	validation_0-logloss:0.50159
[16]	validation_0-logloss:0.50038
[17]	validation_0-logloss:0.49938
[18]	validation_0-logloss:0.49844
[19]	validation_0-logloss:0.49759
[20]	validation_0-logloss:0.49683
[21]	validation_0-logloss:0.49614
[22]	validation_0-logloss:0.49560
[23]	validation_0-logloss:0.49500
[24]	validation_0-logloss:0.49449
[25]	validation_0-logloss:0.49405
[26]	validation_0-logloss:0.49363
[27]	validation_0-logloss:0.49326
[28]	validation_0-logloss:0.49293
[29]	validation_0-loglos

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.1, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=200,
              n_jobs=None, num_parallel_tree=None, random_state=42, ...)

In [19]:
# 검증 데이터 ROC-AUC 평가
val_pred_proba = xgb_model.predict_proba(x_val_enc)[:, 1]
val_roc_auc = roc_auc_score(y_val, val_pred_proba)
print("XGBoost Validation ROC-AUC:", val_roc_auc)

XGBoost Validation ROC-AUC: 0.7361342354552496


In [ ]:
#GridSearchCV를 활용한 하이퍼파라미터 튜닝 (마지막에 해보기. 디버깅문제)
"""
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [4, 6, 8],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}
grid_search = GridSearchCV(
    estimator=XGBClassifier(random_state=42, eval_metric='logloss'),
    param_grid=param_grid,
    scoring='roc_auc',
    cv=3,
    n_jobs=-1,
    verbose=1
)"""

In [20]:
#전체 학습 데이터 전처리 적용
x_full_enc = x.copy()
x_full_enc[categorical_columns] = ordinal_encoder.transform(x_full_enc[categorical_columns])  # fit X
x_full_enc[numeric_columns] = imputer.transform(x_full_enc[numeric_columns])  # fit X

In [21]:
#최종 XGBoost 모델 (단, early stopping 없이 기본 n_estimators 사용)
model_full = XGBClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=6,
    random_state=42,
    eval_metric='logloss'
)
model_full.fit(x_full_enc, y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.1, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=100,
              n_jobs=None, num_parallel_tree=None, random_state=42, ...)

In [22]:
#테스트 데이터 전처리 및 예측
x_test_enc = test.copy()
x_test_enc[categorical_columns] = ordinal_encoder.transform(x_test_enc[categorical_columns])
x_test_enc[numeric_columns] = imputer.transform(x_test_enc[numeric_columns])
test_pred_proba = model_full.predict_proba(x_test_enc)[:, 1]

In [23]:
# 제출 파일 생성
sample_submission = pd.read_csv('./sample_submission.csv')
sample_submission['probability'] = test_pred_proba
sample_submission.to_csv('./0217.01.csv', index=False)
print("Finish! Submission file saved as 0217.01.csv")

Finish! Submission file saved as 0217.01.csv
